# 获取数据

In [ ]:
import os
import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL=DOWNLOAD_ROOT + HOUSING_PATH +"/housing.tgz"
def fetch_housing_data(housing_url=HOUSING_URL,housing_path=HOUSING_PATH):
    '''从网络获取数据下载到本地目录'''
    housing_path=HOUSING_PATH
    housing_url=HOUSING_URL
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url,tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
    '''从文件读取数据并返回dataframe对象'''
    csv_path=os.path.join(housing_path,"housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
#fetch_housing_data()
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
def split_train_test(data,test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data)*test_ratio)
    test_indices=shuffled_indices[:test_set_size]
    train_indices=shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

train_set,test_set=split_train_test(housing,0.2)
print (len(train_set),"train+",len(test_set),"test")

In [ ]:
import hashlib
def test_set_check(identifier,test_ratio,hash):
    return hash(np.int64(identifier)).digest()[-1]<256*test_ratio

def split_train_test_by_id(data,test_tatio,id_column,hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_:test_set_check(id_,test_tatio,hash))
    return data.loc[~in_test_set],data.loc[in_test_set]

#方法一：使用行索引作为唯一标识符，需要确保在数据集末尾添加新数据，并且不会删除任何行
housing_with_id = housing.reset_index()
#方法二：使用经纬度作为唯一标识符，最稳定的特征
housing_with_id["id"] = housing["longitude"]*1000+housing["latitude"]
train_set,test_set = split_train_test_by_id(housing_with_id,0.2,"id")

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split


housing['income_cat']=np.ceil(housing['median_income']/1.5)
housing['income_cat'].where(housing['income_cat']<5,5.0,inplace=True)

train_set,test_set=train_test_split(housing,test_size=0.2,random_state=42)
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(housing,housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
print(housing['income_cat'].value_counts()/len(housing))
print(strat_test_set['income_cat'].value_counts()/len(strat_test_set))
print(test_set['income_cat'].value_counts()/len(strat_test_set))

for set in (strat_train_set,strat_test_set):
    set.drop(["income_cat"],axis=1,inplace=True)

# 从数据探索和可视化获得洞见

In [ ]:
#将地理数据可视化
housing = strat_train_set.copy()
housing.plot(kind='scatter',x='longitude',y='latitude',alpha=0.1,
            s=housing['population']/100,label='population',
            c='median_house_value',cmap=plt.get_cmap('jet'),colorbar=True)
plt.legend()

In [ ]:
#寻找相关性
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes =['median_house_value','median_income','total_rooms','housing_median_age']
scatter_matrix(housing[attributes],figsize=(12,8))

In [ ]:
housing.plot(kind='scatter',x='median_income',y='median_house_value',alpha=0.1)

In [ ]:
#试验不同属性的组合
housing['rooms_per_household']=housing['total_rooms']/housing['households']#家庭每户平均房间数
housing['bedrooms_per_room']=housing['total_bedrooms']/housing['total_rooms']#卧室占比
housing['population_per_household']=housing['population']/housing['households']#每个家庭人口数
corr_matrix=housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

# 机器学习算法的数据准备

In [ ]:
#数据清理
from sklearn.impute import SimpleImputer

housing = strat_train_set.drop("median_house_value",axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

median=housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median)

imputer = SimpleImputer(strategy="median")
housing_num=housing.drop("ocean_proximity",axis=1) #创建一个没有文本属性的数据副本
imputer.fit(housing_num)
print(imputer.statistics_) 
print(housing_num.median().values)
X=imputer.transform(housing_num)
housing_tr = pd.DataFrame(X,columns=housing_num.columns)#将清洗完的ndarray转回dataframe

#处理文本和分类属性
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

#文本转数值编码
encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
print(encoder.classes_)

#数值编码转独热编码
encoder =OneHotEncoder(categories='auto')
#返回一个scipy稀疏矩阵，用于存储大量0
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1)) 

#文本转独热编码
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
print(housing_cat_1hot)

#自定义转换器
from sklearn.base import BaseEstimator,TransformerMixin
rooms_ix,bedrooms_ix,population_ix,household_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
    def __init__(self,add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self,X,y = None):
        return self
    def transform(self,X,y = None):
        rooms_per_household = X[:,rooms_ix]/X[:,household_ix]
        population_per_household = X[:,population_ix]/X[:,household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_ix]/X[:,rooms_ix]
            return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room]
        else:
            return np.c_[X,rooms_per_household,population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
#转换流水线-处理数值
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])
housing_num_tr = num_pipeline.fit_transform(housing_num)

#完整处理数值和分类属性的流水线
from sklearn.pipeline import FeatureUnion
num_attribs=list(housing_num)
cat_attribs=["ocean_proximity"]

class DataFrameSelector(BaseEstimator,TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names=attribute_names
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.attribute_names].values

class MyLabelBinarizer(BaseEstimator,TransformerMixin):
    '''
    使用LabelBinarizer导致参数报错问题
    https://stackoverflow.com/questions/46162855/fit-transform-takes-2-positional-arguments-but-3-were-given-with-labelbinarize'''
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)
    
num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_attribs)),
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('label_binarizer',MyLabelBinarizer()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipline",num_pipeline),
    ("cat_pipline",cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.shape

# 选择和训练模型

In [ ]:
#训练和评估训练集
#使用线性回归模型进行拟合
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

#查看预测值和实际值
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:\t",lin_reg.predict(some_data_prepared))
print("Labels:\t\t",list(some_labels))

#使用均方误差MSE来评估效果
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels,housing_predictions)

#使用均方根误差rmse来观测误差大小
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

均方误差MSE:(线性回归的代价函数)
$$\frac 1m\sum_{i=1}^m(y_i-\hat y_i)^2$$


数据结果欠拟合，需要增加信息完备性-新增特征，或者换用更强的模型

In [ ]:
#使用决策树回归模型进行拟合
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)
housing_predctions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predctions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

In [ ]:
#使用交叉验证来更好的进行评估

#决策树K折交叉验证
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg,housing_prepared,housing_labels,
                        scoring="neg_mean_squared_error",cv=10)
tree_rmse_scores=np.sqrt(-scores)

def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Standard deviation:",scores.std())
    
display_scores(tree_rmse_scores)

#线性回归K折交叉验证
lin_scores = cross_val_score(lin_reg,housing_prepared,housing_labels,
                        scoring="neg_mean_squared_error",cv=10)
lin_rmse_scores=np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

结论：决策树模型严重过拟合，以至于在测试集上表现的比线性回归还要糟糕

试试集成模型-随机森林

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=10)
forest_reg.fit(housing_prepared,housing_labels)
housing_predctions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels,housing_predctions)
forest_rmse = np.sqrt(forest_mse)
scores = cross_val_score(forest_reg,housing_prepared,housing_labels,
                        scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores=np.sqrt(-scores)

print(forest_rmse)

display_scores(forest_rmse_scores)

尝试各种机器学习模型-直到筛出2-5个有效模型

序列化存储

from sklearn.externals import joblib

joblib.dump(forest_reg,"my_model.pkl")

my_model_loaded =joblib.load("my_model.pkl")

# 微调模型 

In [ ]:
#网格搜索
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg,param_grid,cv=5,
                          scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared,housing_labels)
print(grid_search.best_params_)
print(grid_search.best_estimator_)
cvers = grid_search.cv_results_

for mean_score,params in zip(cvers["mean_test_score"],cvers["params"]):
    print(np.sqrt(-mean_score),params)

In [ ]:
#随机搜索 RandomizedSearchCV
#集成方法 随机森林
#分析最佳模型及其错误
feature_importances =grid_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold","pop_per_hhold","bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances,attributes),reverse=True)


In [ ]:
#通过测试集评估系统
final_model =grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value",axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test,final_predictions)
final_rmse = np.sqrt(final_mse)
print (final_rmse)


# 启动、监控和维护系统